In [17]:
class D:
	def __get__(self, instance, owner):
		print(f"{self}.__get__({instance}, {owner})")
		return self

class A:
	def f(self):
		print(f"{self} (type: {type(self)})")

	g = D()


In [20]:
a = A()
a.g

<__main__.D object at 0x7f1b4c403a60>.__get__(<__main__.A object at 0x7f1b4f592e80>, <class '__main__.A'>)


In [21]:
a.__dict__['g'] = 10
a.g

10

In [13]:
a = A()
a.f

<bound method A.f of <__main__.A object at 0x7f1b4c453df0>>

In [14]:
a.f.__get__(A(), A)

<bound method A.f of <__main__.A object at 0x7f1b4c453df0>>

In [16]:
A.f.__get__(A)()

<class '__main__.A'> (type: <class 'type'>)


In [19]:
A().g

<__main__.D object at 0x7f1b4c403a60>.__get__(<__main__.A object at 0x7f1b4c453760>, <class '__main__.A'>)
